In [ ]:
!pip install yelpapi  

     |████████████████████████████████| 127 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install requests  

In [ ]:
!python -m pip install boto3         

In [ ]:
!pip install ipdb

In [ ]:
!pip install simplejson

In [ ]:
from collections import defaultdict
import requests
import csv
import time
from datetime import datetime
import ipdb
from decimal import *
import simplejson as json
import json
import boto3



def check_empty(input):
	if len(str(input)) == 0:
		return 'N/A'
	else:
		return input



dynamodb = boto3.resource('dynamodb',region_name='us-east-1',aws_access_key_id= 'Give access key',aws_secret_access_key= 'Give secret access key')
table = dynamodb.Table('yelp-restaurants')

#define api key, define the endpoint, and define the header
API_KEY = 'Give your own api key generated by yelp'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
ENDPOINT_ID = 'https://api.yelp.com/v3/businesses/' # + {id}
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

#define parameters
PARAMETERS = {'term': 'food', 
			  'limit': 50,
			  'radius': 15000,
			  'offset': 200,
			  'location': 'Manhattan'}

cuisines = ['italian', 'chinese', 'mexican', 'american', 'japanese', 'pizza', 'healthy', 'brunch', 'korean', 'thai', 'vietnamese', 'indian', 'seafood', 'dessert']

manhattan_nbhds = ['Lower East Side, Manhattan',
                   'Upper East Side, Manhattan',
                   'Upper West Side, Manhattan',
                   'Washington Heights, Manhattan',
                   'Central Harlem, Manhattan',
                   'Chelsea, Manhattan',
                   'Manhattan',
                   'East Harlem, Manhattan',
                   'Gramercy Park, Manhattan',
                   'Greenwich, Manhattan',
                   'Lower Manhattan, Manhattan',
                   'Columbus Circle, Manhattan',
                   'Times Square, Manhattan',
                   'Hells Kitchen, Manhattan',
                   'Midtown, Manhattan',
                   'Union Square, Manhattan']

start = time.time()
for nbhd in manhattan_nbhds:
	PARAMETERS['location'] = nbhd
	for cuisine in cuisines: 
		PARAMETERS['term'] = cuisine
		
		#make request to yelp API for specified cuisine + location
		response = requests.get(url = ENDPOINT, params =  PARAMETERS, headers=HEADERS)
		business_data = response.json()['businesses']
		for business in business_data:
			now = datetime.now()
			restauraunt_data = {}
			dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
			# #NOTE: to export into dynamoDB, you cannot have float values + u need to load it as a json file; simplejson library supports Decimal() function
			# #make request to yelp api for specific business ID and get its hours of operation 
			table.put_item(
			Item = {
				'Business_ID':check_empty(business['id']),
				'insertedAtTimestamp': check_empty(dt_string),
				'Name':  check_empty(business['name']),
				'Cuisine': check_empty(cuisine),
				'Rating': check_empty(Decimal(business['rating'])),
				'Number of Reviews' : check_empty(Decimal(business['review_count'])),
				'Address': check_empty(business['location']['address1']),
				'Zip Code': check_empty(business['location']['zip_code']),
				'Latitude': check_empty(str(business['coordinates']['latitude'])),
				'Longitude': check_empty(str(business['coordinates']['longitude'])),
				'Open': 'N/A'
			 }
			)
			#ipdb.set_trace()


	print('Finish',nbhd, time.time()- start)

Fin  Columbus Circle, Manhattan 34.90137243270874
Fin  Times Square, Manhattan 69.11443614959717
Fin  Hells Kitchen, Manhattan 137.89527344703674
Fin  Midtown, Manhattan 207.90347838401794
Fin  Union Square, Manhattan 279.78417110443115


In [ ]:
!pip install requests_aws4auth

  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.
elastic-transport 8.0.1 requires urllib3<2,>=1.26.2, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install elasticsearch==7.13.4

Found existing installation: elasticsearch 8.0.1
Uninstalling elasticsearch-8.0.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/elasticsearch-8.0.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/elasticsearch/*
Proceed (y/n)? y
  Successfully uninstalled elasticsearch-8.0.1
     |████████████████████████████████| 356 kB 5.6 MB/s 


In [ ]:
import boto3
from elasticsearch import Elasticsearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth


def put_into_elasticsearch():
    host = "search-elasticsearch-new-zxhkvgmawqxqlihjppkws6ixly.us-east-1.es.amazonaws.com"
    region = "us-east-1"
    service = "es"
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth('Give access key', 'Give secret access key, 'us-east-1', service)

    es = Elasticsearch(
        hosts=[{'host': host, 'port': 443}],
        http_auth=('user', 'password'),
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection
    )
    dynamodb = boto3.resource('dynamodb',region_name='us-east-1',aws_access_key_id= 'Give access key',aws_secret_access_key= 'Give secret access key')
    table = dynamodb.Table("yelp-restaurants")
    response = None
    while True:
        if response is None:
            response = table.scan()
        else:
            # Scan from where you stopped previously.
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        counter = 0
        for business in response['Items']:
            if not response:
                # Scan from the start.
                response = table.scan()
            restaurantID = business["Business_ID"]
            doc = {
                "Business_ID": restaurantID,
                "Cuisine": business["Cuisine"]
            }
            es.index(
                index="restaurants",
                doc_type="Restaurant",
                id=restaurantID,
                body=doc,
            )
            check = es.get(index="restaurants", doc_type="Restaurant", id=restaurantID)
            if check["found"]:
                print("Index %s succeeded" % restaurantID)
            counter = counter + 1

put_into_elasticsearch()

Index mn5e_496sVYOHxPpKYBtUQ succeeded
Index qAfpHbb4ww_VAGEgTL4B3A succeeded
Index ykXZyQBXxoOMoZfaZMnHmg succeeded
Index uaFHoq-a5XqxF-bsOK9_Qg succeeded
Index IhSVn0TaX8xXb3wcQ-fgcA succeeded
Index cJfbydHniqWsGL2VHVDHEg succeeded
Index rpTW8Ozm8q8kN5RWOYSvFA succeeded
Index 4mzogOLOVDPBDVTO4cz9fA succeeded
Index FXhSEZfIqjMZh4hR0_8jMQ succeeded
Index StFf9fVnAlhwrUWan0C4Ww succeeded
Index VC1udoc_sbHdFaBr0-bMsA succeeded
Index xlv_CcDXH1hkG4Wq5mECkQ succeeded
Index Ry9w-jFALWYhazi6jjyIKw succeeded
Index pqebqpVTT8bR_s9iEu8dOA succeeded
Index HDoJIH-qW9FqJ5WB-bKTqA succeeded
Index BtsXojrMtfLRhhFK7rm35A succeeded
Index _NCh8Gxnrp33NOaDRCxRbQ succeeded
Index fjEtp-z1094fCTzL_tMDAw succeeded
Index xbrQ2dHuQoFJmNxAuOky4A succeeded
Index PqqnNrUtU7XTKQvUM4cyqQ succeeded
Index T1kO6d-QY9i3f4D_yNsU6Q succeeded
Index FkA9aoMhWO4XKFMTuTnl4Q succeeded
Index csVEpHy5KJoFvM1qp5XwLQ succeeded
Index vyyJNMgVo5b3Lyex5GawHg succeeded
Index 6ztuhNteBxQB334A98iP8g succeeded
Index Q9F2ocrmYuGt1yn3M7M

KeyError: ignored